In [7]:
# Yes I know this notebook is a mess.

import math
import statistics as st
from statsmodels.stats.multitest import fdrcorrection as fdr
import scipy.stats
import pandas as pd
import numpy as np
from pprint import pprint
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as FF

import matplotlib.pyplot as plt
import seaborn as sns

plotly.offline.init_notebook_mode(connected=True)

DATA_LOC = "cleanData_comma.csv"

# import data to pandas.dataframe
df = pd.read_csv(DATA_LOC)
df.head()

,AD_1,AD_2,AD_3,AD_4,AD_5,AD_6,AD_7,AD_8,AD_9,AD_10,...,N48_37,N48_38,N48_39,N48_40,N48_41,N48_42,N48_43,N48_44,N48_45,N48_46
0,0.49,0.05,0.58,0.21,3.35,5.44,6.38,0.53,1.77,0.15,...,1.28,1.67,1.88,1.60,1.19,2.73,0.98,1.41,0.71,0.16
1,1.77,0.16,1.45,0.29,8.60,7.01,11.25,3.39,2.38,0.14,...,1.28,1.37,2.20,1.95,1.42,3.82,1.32,2.60,0.68,0.34
2,2.55,0.20,1.97,0.49,10.60,9.67,13.52,1.29,2.61,0.35,...,1.66,1.79,1.07,1.44,1.20,2.88,1.85,2.16,0.70,0.12
3,3.05,0.39,1.98,0.55,10.18,11.81,13.68,1.05,2.74,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.69,0.26,1.70,0.39,8.90,9.06,12.31,0.88,2.29,0.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def prepare_matrix(results_data_dict, filtered=False):
    head = ["", "T-Stat", "p-val"]
    bulk = []
    # test data
    for i,j in results_data_dict.items():
        counter = 1
        for res in j.values():
            name = i + " peak" + str(counter)
            if filtered == True:
                    if res[1]<0.01:
                        bulk.append([name, res[0], res[1]]) # res[0] is Tstat, res[1] is p-val
            else:
                bulk.append([name, res[0], res[1]]) # res[0] is Tstat, res[1] is p-val
            counter+=1
    return [head] + sorted(bulk, key=lambda x: x[1], reverse=True)

def prepare_bar_data(dataframe):
    # get means
    ad_mean = []
    n24_mean = []
    n48_mean = []
    for i in range(1,47):
        ad_mean.append(round(st.mean([i for i in df["AD_"+str(i)].values if not np.isnan(i)]), 3))
        n24_mean.append(round(st.mean([i for i in df["N24_"+str(i)].values if not np.isnan(i)]), 3))
        n48_mean.append(round(st.mean([i for i in df["N48_"+str(i)].values if not np.isnan(i)]), 3))
    # get errors
    return ad_mean, n24_mean, n48_mean

In [ ]:
# prepare table headers
ads = []
n24s = []
n48s = []
col_names = []
for i in range(1, 47):
    ads.append("AD_vs_24_PEAK_"+str(i))
    n24s.append("AD_vs_48_PEAK"+str(i))
    #n48s.append("N48_"+str(i))

col_names = ads + n24s

In [6]:
# CROATIAN VERSION
ads = []
n24s = []
n48s = []
col_names = []
for i in range(1, 47):
    ads.append("Frakcija "+str(i))
    n24s.append("Frakcija "+str(i))
    #n48s.append("N48_"+str(i))

col_names = ads + n24s

In [3]:
results = {
    "AD_vs_N24": {},
    "AD_vs_N48": {},
    #"N24_vs_N48": {}
}

# automate T-Testing
for i in range(1,47):
    # test ADULT<i> vs N24<i>
    ADi_logged = [math.log2(j) for j in [i/100 for i in df["AD_"+str(i)].values if not np.isnan(i)]]
    N24i_logged = [math.log2(j) for j in [i/100 for i in df["N24_"+str(i)].values if not np.isnan(i)]]
    twosampleT_result = scipy.stats.ttest_ind(ADi_logged, N24i_logged)
    results["AD_vs_N24"][i] = twosampleT_result
    
    # test ADULT<i> vs N48<i>
    ADi_logged = [math.log2(j) for j in [i/100 for i in df["AD_"+str(i)].values if not np.isnan(i)]]
    N48i_logged = [math.log2(j) for j in [i/100 for i in df["N48_"+str(i)].values if not np.isnan(i)]]
    twosampleT_result = scipy.stats.ttest_ind(ADi_logged, N48i_logged)
    results["AD_vs_N48"][i] = twosampleT_result
    
    # test N24<i> vs N48<i>
#     N24i_logged = [math.log2(i) for i in df["N24_"+str(i)].values if not np.isnan(i)]
#     N48i_logged = [math.log2(i) for i in df["N48_"+str(i)].values if not np.isnan(i)]
#     twosampleT_result = scipy.stats.ttest_ind(N24i_logged, N48i_logged)
#     results["N24_vs_N48"][i] = twosampleT_result    

In [4]:
# show t-test results
tests_matrix = prepare_matrix(results, True)
twosample_table = FF.create_table(tests_matrix, index=True)
plotly.offline.iplot(twosample_table, filename='T-test results')

In [ ]:
print(f"AD vs N24 FOR 16: {results['AD_vs_N24'][16][1]}, mean AD: {st.mean([i for i in df['AD_16'].values if not np.isnan(i)])}, mean 24: {st.mean([i for i in df['N24_16'].values if not np.isnan(i)])}")
print(f"AD vs N48 FOR 16: {results['AD_vs_N48'][16][1]}, mean: {st.mean([i for i in df['AD_16'].values if not np.isnan(i)])}, mean 48 {st.mean([i for i in df['N48_16'].values if not np.isnan(i)])}")

In [7]:
# apply multiple testing correction
def prep_correction(results_data_dict):
    pvals = []
    # test data
    for i,j in results_data_dict.items():
        counter = 1
        for res in j.values():
            pvals.append(res[1]) # res[0] is Tstat, res[1] is p-val
            counter+=1
    return pvals

all_p_val = prep_correction(results)
corrected = fdr(all_p_val)
res_significant = []
for i in range(len(col_names)):
    if corrected[0][i]:
        res_significant.append([col_names[i], round(corrected[1][i], 3)]) #, corrected[0][i]])

        # engl version
# prep_for_table = [["Peak", "Adjusted p-val", "Significant"]] + res_significant

#for Cro ver
prep_for_table = [["ODRASLI vs N24h", "Korigirana p-vrijednost (FDR)"]] + res_significant[0:12]

sig_table = FF.create_table(prep_for_table, index=False)
sig_table.layout.width=450
plotly.offline.iplot(sig_table, filename='FDR corr results')


In [8]:
prep_for_table = [["ODRASLI vs N48h", "Korigirana p-vrijednost (FDR)"]] + res_significant[12:]

sig_table = FF.create_table(prep_for_table, index=False)
sig_table.layout.width=450
plotly.offline.iplot(sig_table, filename='FDR corr results')

In [ ]:
# t-test oligomannose contents
avg_oligo_ad = [
    15.22,16.215, 19.145, 20.28, 17.76,
]
avg_oligo_48 = [
    13.795, 12.305 ,12.18,
]
avg_oligo_24 = [
    11.68, 14.335, 14.125, 14.125,
]

oligo_tt_AD_v_48 = scipy.stats.ttest_ind(avg_oligo_ad, avg_oligo_48)
oligo_tt_AD_v_24 = scipy.stats.ttest_ind(avg_oligo_ad, avg_oligo_24)

print(f"OLIGO RES:\n\tp(AD v 24): {oligo_tt_AD_v_24.pvalue}\n\tp(AD v 48): {oligo_tt_AD_v_48.pvalue}\npval sig <{0.05/2}")


In [ ]:
# Draw oligomannose content barplots

width = 0.5
plot_data = [st.mean(i) for i in [avg_oligo_ad, avg_oligo_48, avg_oligo_24]]
ind = np.arange(len(plot_data))
ticks = ["Odrasli", "N48", "N24"]
sns.set_context("talk")
sns.set_style("white",
    {
    'xtick.bottom': True,
    'ytick.left': True,
    'patch.edgecolor': 'b',
    }
)
fig, ax = plt.subplots()
oligo_bars = ax.bar(ind, plot_data, width)
plt.xticks(ind, ticks)
plt.ylabel('% Ukupne površine')
plt.title('Udio Man(5-9)GlcNAc2')
plt.ylim(ymax=25)
for bar in oligo_bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., 1.05*height,
                f"{round(height,1)}%",
                ha='center', va='bottom')

plt.show()
fig.savefig("oligoman_content.pdf")

In [10]:
# calculate effect sizes
sigs_AD_N24 = [3,4,5,6,7,16,24,25,27,32,34,39]
sigs_AD_N48 = [3,5,6,7,9,11,16,24,27,31,34]

def calc_cohen_d(mean1,mean2,sdev1,sdev2):
    return mean1 - mean2 / (math.sqrt((sdev1 ** 2 + sdev2 ** 2) / 2))

# this can be rewritten to use array-like objs and calc everything from there
# but not right now...
# also this is heavily bound...
def calc_hedges_g(mean1, mean2, sdev1, sdev2, size1, size2):
    return ((mean1-mean2)/ math.sqrt(((size1-1)*sdev1**2+(size2-1)*sdev2**2)/(size1-1+size2-1))) * \
            ((size1+size2-3)/(size1+size2-2.25)) * math.sqrt((size1+size2-2)/(size1+size2))

    
def calc_eff_size(sig_peaks_list, group2): 
    """Groups: AD_, N24_, N48_"""
    if group2 not in ["N24_", "N48_"]:
        raise ValueError("Invalid group. Use AD_, N24_, N48_")
    res = {}
    s_size = {
        "AD_": 5,
        "N24_":4,
        "N48_":3
    }
    for i in sig_peaks_list:
        data_g1 = [i for i in df["AD_"+str(i)].values if not np.isnan(i)] # always adult
        mean_g1 = st.mean(data_g1)
        sdev_g1 = st.stdev(data_g1)
        
        data_g2 = [i for i in df[group2+str(i)].values if not np.isnan(i)] # one of the neonates
        mean_g2 = st.mean(data_g2)
        sdev_g2 = st.stdev(data_g2)
        if group2 == "N24_":
            res[i] = calc_hedges_g(mean_g1, mean_g2, sdev_g1, sdev_g2, 5, 4)
        if group2 == "N48_":
            res[i] = calc_hedges_g(mean_g1, mean_g2, sdev_g1, sdev_g2, 5, 3)
    return res


ad_v_24_effs = calc_eff_size(sigs_AD_N24, "N24_")
ad_v_48_effs = calc_eff_size(sigs_AD_N48, "N48_")

pprint(ad_v_24_effs)

pprint(ad_v_48_effs)

{3: 2.220629421555877,
 4: 1.9626084810944173,
 5: 2.2208670949765947,
 6: 2.0137451178535426,
 7: 1.7763564954378868,
 16: 2.768335321188477,
 24: -2.4192682506255934,
 25: -1.977635265507001,
 27: -2.9153263937925353,
 32: -2.338017976580972,
 34: -1.8783305745834196,
 39: -2.1021757277224995}
{3: 1.9679872875805946,
 5: 2.242948690250535,
 6: 2.013029840243192,
 7: 1.9428151147106978,
 9: 2.2522639612625768,
 11: 1.9373813981041612,
 16: 2.4569308878805947,
 24: -2.5950366204703577,
 27: -2.7935658114041306,
 31: -2.177476283908013,
 34: -2.370405386091018}
